### Loading data

In [ ]:
num_seqs = 0#4
X_am, X_pssm, mask, labels = load_data("test_saliency/", num_seqs)
split_value = 5534
print X_am.shape

In [ ]:
with open("test_saliency/predictions.pkl", 'rb') as f:
    predictions = pickle.load(f)
print predictions.shape

**Check aminoacids coding**

In [ ]:
sorted_fang = sorted(aaMap_fang.keys(), key=lambda letter: aaMap_fang[letter])

aaMap = ['A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P', 'S', 'R', 'T', 'W', 'V', 'Y', 'X']
pssm = 'ACDEFGHIKLMNPQRSTVWXY'
aaMap_jurtz = {amino: i for i, amino in enumerate(aaMap)}
pssm_jurtz = {amino: i for i, amino in enumerate(pssm)}

index_am = np.array([aaMap_jurtz[letter] for letter in sorted_fang if letter is not 'NoSeq'])
index_pssm = np.array([pssm_jurtz[letter] for letter in sorted_fang if letter is not 'NoSeq'])

#aaMap_fang['X']
print len(aaMap)
print len(index_am), len(index_pssm)
print index_am

In [ ]:
seq = 3
pos = 0

print X_am[seq,pos]
print len(X_am[seq,pos])

dc = decode(X_am[seq,:end])
print len(dc)
print dc[pos]
print dc

### Evaluate saliencies

**Aggregate saliencies**

In [ ]:
target_class = "H"
num_seqs = 4
saliencies = np.zeros((2,700,num_seqs,700,21))
for seq in range(num_seqs):
    with open("test_saliency/saliencies/saliencies"+str(seq)+target_class+".pkl", "rb") as f:
        temp = np.array(pickle.load(f))
        saliencies[:,:,seq,:,:] = temp[:,:,0,:,:]
print("Saliencies shape", saliencies.shape)

**Inspect sequential saliency**

In [ ]:
seq = 0
final_pos = 110
window = 29
ini = 80

end = final_pos + window + 1
end_seq = int(sum(mask[seq]))
if end > end_seq:
    end = end_seq
if final_pos > end_seq:
    final_pos = end_seq

preds = convertPredictQ8Result2HumanReadable( predictions[seq] )
    
fig, ax = plt.subplots(figsize=(15,3))
for pos in range(final_pos):
    if preds[pos] == 'H':
    #if ssConvertMap[labels[seq,pos]] == 'H':        
        temp = np.einsum('ij,ij->i', abs(saliencies[0,pos,seq,ini:end,:]), X_am[seq,ini:end])
        temp += np.einsum('ij,ij->i', abs(saliencies[1,pos,seq,ini:end,:]), X_pssm[seq,ini:end])
        ax.plot(temp, marker='.', label=str(pos))
        #ax.plot(np.sum(np.sum(abs(saliencies[:,pos,seq,:end,:]), axis=2), axis=0), marker='.') # Aminoacid importance

ax.xaxis.set(ticks=range(end), ticklabels=[preds[ini+i]+"\n"+ssConvertMap[el]+"\n"+str(ini+i) for i, el in enumerate(labels[seq,ini:end])])
ax.margins(0)
ax.legend()
ax2 = ax.twiny()
ax2.xaxis.set(ticks=range(end-ini), ticklabels=decode(X_am[seq,ini:end]))
""

**Gradient computation drafts**

In [ ]:
#weights = model.trainable_weights # weight tensors
gradients = theano.gradient.grad(model.outputs[0][seq,pos,7], wrt=[model.inputs[0], model.inputs[1]])
gradients = theano.gradient.jacobian(model.outputs[0][seq,pos,5].flatten(), wrt=[model.inputs[0], model.inputs[1]])
#gradients = K.gradients(model.predict, model.inputs)
#gradients = K.gradients(model.total_loss, model.inputs) # gradient tensors
#input_tensors = model.inputs + model.sample_weights + model.targets + [K.learning_phase()]
#get_gradients = K.function(inputs=input_tensors, outputs=gradients)

In [ ]:
get_gradients = K.function(inputs=[model.inputs[0], model.inputs[1], K.learning_phase()], outputs=gradients)

In [ ]:
inputs = [X_test_am[seq,...], X_test_pssm[seq,...], 0]
inputs = [inputs[0][None,...], inputs[1][None,...], inputs[2]]
grads = get_gradients(inputs)
grads = np.array(grads)